In [ ]:
import os
import sys
import json
import copy
import warnings
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yaml
import joblib
import cv2
from tqdm import tqdm
from PIL import Image
from IPython.display import display
from scipy.ndimage import gaussian_filter1d
import wandb

sys.path.append("../")
from src.utils.parser import *

warnings.filterwarnings('ignore')

# Set color palette
sns.set_palette("Set2")
palette = sns.color_palette("Set2", 10)

# Initialize wandb
api = wandb.Api()
try:
    with open(os.path.expanduser("~/.wandb_api_key"), "r") as f:
        wandb_key = f.read().strip()
    wandb.login(key=wandb_key)
except FileNotFoundError:
    print("Warning: wandb API key file not found. Some features may not work.")

%load_ext autoreload
%autoreload 2

In [ ]:
# Configuration paths
test_trajs_pairs_path = "../config/data_split_config/0603_frankarobot_obj20_sg10_persg5/test_traj_pairs.json"
with open(test_trajs_pairs_path, "r") as f:
    test_trajs_pairs = json.load(f)

image_folder = "../data/images/0603_frankarobot_obj20_sg10_persg5"
camera_viewpoint = 0  # Image format: traj_0_view_0.png

In [ ]:
# Load theta sampling data
thetas_sampled_data_path = "../config/humans/thetas_sampled_data_0809.json"
with open(thetas_sampled_data_path, "r") as f:
    thetas_sampled_data = json.load(f)

print("Available keys:", thetas_sampled_data.keys())
test_thetas_30 = thetas_sampled_data["test_thetas_30"]
train_thetas = thetas_sampled_data["sampled_thetas"]

In [ ]:
# Load environment configuration
env_config = "../config/reward_learning/obj20_sg10_persg5/frankarobot_obj20_sg10_persg5_maskedrl_llm_mask_mweight1_mnoise1_hidden128_256_128.yaml"
with open(env_config, "r") as stream:
    params = yaml.safe_load(stream)

# Set random seed
set_seed(12345)

# Make environment
env = make_env(params["env"])

# Load data and split into train and test
indices_file = os.path.join(params["irl"]["data_split_config_path"], "split_indices.json")
all_trajs, train_trajs, test_trajs = load_split_data(
    params["env"]["trajset_file"],
    params["env"]["per_SG"],
    params["env"]["train_test_split"],
    indices_file=indices_file
)
print(f"Total trajectories: {len(all_trajs)}, Train: {len(train_trajs)}, Test: {len(test_trajs)}")
traj_info = params["env"]["trajset_file"].split("/")[-1].split(".")[0]

# Configure human preferences for feature calculation
human_params = {
    "features": ["table", "laptop", "proxemics", "human", "coffee"],
    "feature_scaling": "normalize",
    "preferencer": {
        "theta": [1.0, 1.0, 1.0, 1.0, 1.0],
        "beta": 20.0,
        "f_method": "boltzmann",
        "s_method": "luce"
    },
    "type": params["env"]["type"]
}

human = make_human(human_params, env, train_trajs)
all_features = np.array([human.calc_features(traj) for traj in all_trajs])
print(f"Feature shape: {all_features.shape}")

In [ ]:
def get_theta_key(theta):
    """
    Generate an interpretable key for a given theta vector.
    
    Args:
        theta: A vector with elements in {-1, 0, 1}
        
    Returns:
        String key, e.g., "-1_0_1_1_0" for theta [-1, 0, 1, 1, 0]
    """
    return '_'.join(str(x) for x in theta)


# Load additional configuration if needed
traj_path = "../data/traj_sets/0603_frankarobot_obj20_sg10_persg5.npy"
traj_set = np.load(traj_path, allow_pickle=True)
print(f"Trajectory set loaded from {traj_path}")
print(f"Trajectory set shape: {traj_set.shape}")

# Load human configuration
human_config = "../config/humans/frankarobot_multiple_humans.yaml"
with open(human_config, "r") as stream:
    humans_params_list = yaml.safe_load(stream)

# Load demo indices
seed = 12345
demo_queries = 10
demo_indices_file = os.path.join(params["irl"]["data_split_config_path"], f"demo_indices_100_{seed}.json")
with open(demo_indices_file, "r") as f:
    saved_demo_indices = json.load(f)
    for key in saved_demo_indices.keys():
        saved_demo_indices[key] = saved_demo_indices[key][:demo_queries]
print(f"Loaded demo indices from {demo_indices_file}")

# Load preprocessed trajectory data
preprocessed_traj_data_path = os.path.join(params["irl"]["data_split_config_path"], f"preprocessed_traj_data_{seed}.pkl")
preprocessed_traj_data = joblib.load(preprocessed_traj_data_path)

scaling_coeffs_path = os.path.join(params["irl"]["data_split_config_path"], f"scaling_coeffs_{seed}.pkl")
if os.path.exists(scaling_coeffs_path):
    scaling_coeffs = joblib.load(scaling_coeffs_path)

# Calculate test features
test_features = np.array([human.calc_features(traj) for traj in test_trajs])

In [ ]:
# Verify preprocessed data shapes
print(f"Preprocessed test states shape: {preprocessed_traj_data['test_states'].shape}")
print(f"Preprocessed test features shape: {preprocessed_traj_data['test_features'].shape}")
print(f"Features match: {np.allclose(preprocessed_traj_data['test_features'], test_features)}")

In [ ]:
# Check trajectory pair indices range
test_trajs_pairs_array = np.array(test_trajs_pairs)
print(f"Trajectory pair indices - Min: {test_trajs_pairs_array.min()}, Max: {test_trajs_pairs_array.max()}")

In [ ]:
def annotate_rewards(image, model_name, reward_1, reward_2):
    """
    Annotate the image with the model name and rewards.
    
    Args:
        image: Input image (numpy array)
        model_name: Name of the model
        reward_1: Reward value for first trajectory
        reward_2: Reward value for second trajectory
        
    Returns:
        Annotated image
    """
    # Add margin below the image
    image = cv2.copyMakeBorder(image, 0, 50, 0, 0, cv2.BORDER_CONSTANT, value=[255, 255, 255])
    cv2.putText(image, f"{model_name} Reward", (10, image.shape[0] - 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    cv2.putText(image, f"{reward_1:.2f}", (280, image.shape[0] - 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    cv2.putText(image, f"{reward_2:.2f}", (image.shape[1] - 140, image.shape[0] - 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    return image


# Select test theta (choose from 0 to 29)
test_theta_idx = 0
test_theta = test_thetas_30[test_theta_idx]
print(f"Test theta: {test_theta}")

# Calculate rewards for test trajectories
rewards = test_features.dot(test_theta)
traj_pairs_rewards = []
count_ties = 0

# Visualize pairs of test trajectories
for traj_pair_idx in range(len(test_trajs_pairs)):
    traj_pair = test_trajs_pairs[traj_pair_idx]
    traj_1, traj_2 = traj_pair

    # Calculate rewards
    reward_1 = rewards[traj_1]
    reward_2 = rewards[traj_2]
    traj_pairs_rewards.append([reward_1, reward_2])
    
    if reward_1 == reward_2:
        count_ties += 1

    # Load and resize images
    img_1_path = os.path.join(image_folder, f"traj_{traj_1}_view_{camera_viewpoint}.png")
    img_2_path = os.path.join(image_folder, f"traj_{traj_2}_view_{camera_viewpoint}.png")
    img_1 = cv2.imread(img_1_path)
    img_2 = cv2.imread(img_2_path)
    img_1 = cv2.resize(img_1, (224, 224))
    img_2 = cv2.resize(img_2, (224, 224))
    
    # Create concatenated image with margins
    left_margin = 200
    margin = 50
    img_concat = np.ones(
        (img_1.shape[0], img_1.shape[1] + img_2.shape[1] + margin + left_margin, 3),
        dtype=np.uint8
    ) * 255
    img_concat[:, left_margin:(img_1.shape[1] + left_margin), :] = img_1
    img_concat[:, img_1.shape[1] + margin + left_margin:, :] = img_2
    
    # Add white margin above
    img_concat = cv2.copyMakeBorder(img_concat, 50, 0, 0, 0, cv2.BORDER_CONSTANT, value=[255, 255, 255])
    
    # Add text annotations
    cv2.putText(img_concat, f"Pair {traj_pair_idx + 1}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    cv2.putText(img_concat, f"Traj {traj_1}", (60 + left_margin, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    cv2.putText(img_concat, f"Traj {traj_2}", (img_1.shape[1] + margin + 60 + left_margin, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    
    # Annotate with rewards
    img_concat = annotate_rewards(img_concat, "GT", reward_1, reward_2)
    
    # Display image
    display(Image.fromarray(cv2.cvtColor(img_concat, cv2.COLOR_BGR2RGB)))

print(f"Number of ties: {count_ties} out of {len(test_trajs_pairs)}")
traj_pairs_rewards = np.array(traj_pairs_rewards)
print(traj_pairs_rewards)

# Sample a new test traj set from a large traj set

In [ ]:
all_trajs_path = "../data/traj_sets/0815_frankarobot_obj100_sg50_persg50_dict.npy"
all_trajs = np.load(all_trajs_path, allow_pickle=True).item()
print("Number of Object configurations:", len(all_trajs))
print(all_trajs[0].keys())

In [ ]:
all_trajs[0]["object_centers"], all_trajs[0]["trajs"].shape
# output is
# ({'HUMAN_CENTER': [-0.35557814847495184, -0.696818238823879, 0.9],
#   'LAPTOP_CENTER': [-0.663542735335324, -0.144649949824222, 0.635],
#   'TABLE_CENTER': [-0.65, 0.0, 0.0]},
#  (2500, 21, 121))
# 2500 is 50 start goal pairs, 50 trajs per pair
# 100 object configurations

test_trajs_pairs_indices = []
test_trajs_set = []
max_diffs = []

# for theta_idx in range(len(test_thetas_30)):
#     theta = test_thetas_30[theta_idx]
for theta_idx in range(len(train_thetas)):
    theta = train_thetas[theta_idx]
    print("Processing theta:", theta, "with index", theta_idx)
    theta_key = get_theta_key(theta)
    test_trajs_pairs_indices_per_theta = []
    for i in tqdm(range(len(all_trajs))):
        # for each object configuration,
        trajs = all_trajs[i]["trajs"]
        # reshape this into 50 x 50 x 21 x 121
        trajs = trajs.reshape(50, 50, 21, 121)
        # find the pair with highest reward difference
        max_diff = -np.inf
        max_pair = None
        for j in range(trajs.shape[0]):
            # for each start goal pair, first calculate reward for each trajectory
            # then, choose the trajectory pair with highest reward difference
            features = np.array([human.calc_features(traj) for traj in trajs[j]])
            rewards = features.dot(theta)
            
            for k in range(trajs.shape[1]):
                for l in range(k + 1, trajs.shape[1]):
                    diff = abs(rewards[k] - rewards[l])
                    if diff > max_diff:
                        max_diff = diff
                        max_pair = (k, l, j)
        if max_pair is not None:
            # add the pair to the list
            test_trajs_pairs_indices_per_theta.append([len(test_trajs_set), len(test_trajs_set) + 1])
            test_trajs_set.append(trajs[max_pair[2]][max_pair[0]])
            test_trajs_set.append(trajs[max_pair[2]][max_pair[1]])
            # print(f"Object configuration {i}, theta {theta_key}, pair {max_pair} with max diff {max_diff:.2f} added.")
            max_diffs.append(max_diff)
    # break
    # add the pairs for this theta to the main list
    test_trajs_pairs_indices.append(test_trajs_pairs_indices_per_theta)
# save the test trajectory pairs indices
test_trajs_pairs_indices_path = "../config/data_split_config/0815_frankarobot_obj100_sg50_persg50/seen_theta_test_traj_pairs_indices.json"
if not os.path.exists(os.path.dirname(test_trajs_pairs_indices_path)):
    os.makedirs(os.path.dirname(test_trajs_pairs_indices_path))
json.dump(test_trajs_pairs_indices, open(test_trajs_pairs_indices_path, "w"))
print("Test trajectory pairs indices saved to", test_trajs_pairs_indices_path)
# save the test trajectory set
test_trajs_set_path = "../config/data_split_config/0815_frankarobot_obj100_sg50_persg50/seen_theta_test_trajs_set.npy"
test_trajs_set = np.array(test_trajs_set)
np.save(test_trajs_set_path, test_trajs_set)
print("Test trajectory set saved to", test_trajs_set_path)
                
print("Number of test trajectory pairs:", len(test_trajs_pairs_indices))

In [ ]:
# test_trajs_set.shape
# reshape into -1, 2, 21, 121 and sample 100 pairs
# repeat this 10 times, save as different test sets
# set different random seeds for that
test_trajs_set = np.array(test_trajs_set)
print("Test trajectory set shape:", test_trajs_set.shape)
test_trajs_set_as_pairs = test_trajs_set.reshape(-1, 2, 21, 121)
print("Test trajectory set as pairs shape:", test_trajs_set_as_pairs.shape)

for random_seed in [12345, 54321, 11111, 22222, 33333, 44444, 55555, 66666, 77777, 88888]:
    print("Sampling test trajectory set with random seed:", random_seed)
    np.random.seed(random_seed)
    sampled_indices = np.random.choice(test_trajs_set_as_pairs.shape[0], size=100, replace=False)
    sampled_test_trajs_set = test_trajs_set_as_pairs[sampled_indices]
    # print(f"Random seed {random_seed}, sampled indices: {sampled_indices}")
    # print(f"Sampled test trajectory set shape: {sampled_test_trajs_set.shape}")
    # save the sampled test trajectory set
    sampled_test_trajs_set_path = f"../config/data_split_config/0815_frankarobot_obj100_sg50_persg50/seen_theta_test_trajs_set_100_{random_seed}.npy"
    if not os.path.exists(os.path.dirname(sampled_test_trajs_set_path)):
        os.makedirs(os.path.dirname(sampled_test_trajs_set_path))
    np.save(sampled_test_trajs_set_path, sampled_test_trajs_set)
    print("Sampled test trajectory set saved to", sampled_test_trajs_set_path)
    # break

In [ ]:
np.array(test_trajs_set).shape
# np.array(test_trajs_pairs_indices).shape
# len(all_trajs)
# trajs.shape
test_trajs_pairs_indices
# test_trajs_pairs_indices_per_theta

In [ ]:
# max_diffs.max(), 
# distribution of max_diffs
plt.figure(figsize=(10, 5))
sns.histplot(max_diffs, bins=30, kde=True)
plt.title("Distribution of Maximum Reward Differences for Each Theta")

In [ ]:
# calculate rewards for all_trajs, train_trajs, test_trajs
all_trajs_list = []
for i in range(len(all_trajs)):
    trajs = all_trajs[i]["trajs"]
    # reshape this into 50 x 50 x 21 x 121
    all_trajs_list.append(trajs)
all_trajs_list = np.concatenate(all_trajs_list, axis=0)
all_features = np.array([human.calc_features(traj) for traj in all_trajs_list])
train_features = np.array([human.calc_features(traj) for traj in train_trajs])
test_features = np.array([human.calc_features(traj) for traj in test_trajs])
# calculate rewards for all features
# plot the distribution of rewards for all features. one plot per all, train, test. for each plot, use 5 subplots 
# for each feature
# def plot_feature_rewards(features, rewards, feature_names, title):
#     """
#     Plot the distribution of rewards for each feature.
#     """
#     num_features = features.shape[1]
#     fig, axes = plt.subplots(1, num_features, figsize=(20, 5))
#     for i in range(num_features):
#         sns.histplot(rewards[features[:, i] > 0], ax=axes[i], kde=True)
#         axes[i].set_title(f"{feature_names[i]} (mean: {np.mean(rewards[features[:, i] > 0]):.2f})")
#         axes[i].set_xlabel("Reward")
#         axes[i].set_ylabel("Density")
#     plt.suptitle(title)
#     plt.tight_layout()
#     plt.show()

# feature_names = human_params["features"]
# # plot for all features
# plot_feature_rewards(all_features, all_features.dot(test_thetas_30[0]), feature_names, "All Features Rewards Distribution")
# plot_feature_rewards(train_features, train_features.dot(test_thetas_30[0]), feature_names, "Train Features Rewards Distribution")
# plot_feature_rewards(test_features, test_features.dot(test_thetas_30[0]), feature_names, "Test Features Rewards Distribution")





In [ ]:
def plot_reward_distribution(features, title):
    """
    Plot the distribution of rewards.
    """
    fig, axes = plt.subplots(1, 5, figsize=(10, 5))
    # sns.histplot(rewards, kde=True)
    for i in range(features.shape[1]):
        # draw histplot on each subplot ax
        sns.histplot(features[:, i], ax=axes[i], kde=True)
        axes[i].set_title(f"Feature {i+1} (mean") #: {np.mean(features[:, i]):.2f})")
    plt.title(title)
    plt.xlabel("Reward")
    plt.ylabel("Density")
    plt.show()
# plot for all features
plot_reward_distribution(all_features, "All Features Rewards Distribution")
plot_reward_distribution(train_features, "Train Features Rewards Distribution")
plot_reward_distribution(test_features, "Test Features Rewards Distribution")